# Breast cancer image classification

In [9]:
from IPython.display import display, HTML
from babel.util import missing

display(HTML("<style>.container {widht: 100% !important;}</style>"))

## Import packages

In [10]:
import subprocess
import sys

import os
import shutil
import random
from pathlib import Path

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

# GPU check

In [11]:
if torch.cuda.is_available():
    print("__CUDNN VERSION:", torch.backends.cudnn.version())
    print("Device Name:", torch.cuda.get_device_name(0))
    DEVICE = 'cuda'
else:
    print("CUDA is not available.")
    DEVICE = 'cpu'

torch.device(DEVICE)

__CUDNN VERSION: 91002
Device Name: NVIDIA GeForce RTX 5070


device(type='cuda')

# Export requirements.txt

In [12]:
def export_requirements():
    try:
        result = subprocess.run([sys.executable, "-m", "pip", "freeze"],
                                   capture_output=True,
                                   text=True,
                                   check=True)
        with open('requirements.txt', 'w') as f:
            f.write(result.stdout)
        print('requirements.txt file generated sucessfully.')
    except subprocess.CalledProcessError as e:
        print('error:', e)


export_requirements()

requirements.txt file generated sucessfully.


# Load dataset

### Split dataset

In [15]:
new_split = True

#dataset path
data_og = "data_og"
data_split = 'data_split'

#split ratio
# train + val + test == 1.0
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1


if new_split:
 #create dir
    for split in ["train", "val", "test"]:
        for cls in os.listdir(data_og):
            Path(f"{data_split}/{split}/{cls}").mkdir(parents=True, exist_ok=True)

    #split per class
    for cls in os.listdir(data_og):
        class_path = os.path.join(data_og, cls)
        images = os.listdir(class_path)

        random.shuffle(images)

        n_total = len(images)
        n_train = int(n_total * train_ratio)
        n_val = int(n_total * val_ratio)

        train_images = images[:n_train]
        val_images = images[n_train:n_train + n_val]
        test_images = images[n_train + n_val:]

        def copy_images(img_list, split_name):
            for img in img_list:
                src = os.path.join(class_path, img)
                dst = os.path.join(data_split, split_name, cls, img)
                shutil.copy2(src, dst)

        copy_images(train_images, "train")
        copy_images(val_images, "val")
        copy_images(test_images, "test")

        print(f"Classe '{cls}': {n_train} train | {n_val} val | {len(test_images)} test")


Classe 'benign': 1764 train | 504 val | 253 test
Classe 'maglinant': 3578 train | 1022 val | 512 test
